# Merging the Dataframes


1. set nan values for abstracts and titles
2. merge dataframes
3. max information from abstracts and titles


In [6]:
import pandas as pd
import numpy as np


import sys
import os

# Get the path to the Python executable inside the venv
executable_path = sys.executable

# Get the base path of the venv (two levels up from the executable)
base_path = os.path.dirname((os.path.dirname(os.path.dirname(executable_path))))
print(base_path)

/Users/jlq293/Projects/Study-1-Bibliometrics


### read in pubmed


In [7]:
df_pubmed = pd.read_parquet(
    "../data/processed/medline/medline_data_processed_doisfetched.parquet"
)
print(df_pubmed.shape)

df_pubmed.head(2)

(4478, 11)


,title,year,link,abstract,doi,journal,authors,author_keywords,pubmed_id,document_type,source
0,A comparison of citalopram and paroxetine in t...,2001,https://pubmed.ncbi.nlm.nih.gov/11434404/,Serotonin Selective Re-uptake Inhibitors (SSRI...,10.1055/s-2001-14283,Pharmacopsychiatry,"[Perna, G, Bertani, A, Caldirola, D, Smeraldi,...",[],11434404,Clinical Trial,PubMed
1,"Chronic schizophrenia: response to clozapine, ...",1997,https://pubmed.ncbi.nlm.nih.gov/9090343/,None,10.1176/ajp.154.4.543,The American journal of psychiatry,"[Patel, J K, Salzman, C, Green, A I, Tsuang, M T]",[],9090343,Case Reports,PubMed


In [8]:
df_pubmed.columns

Index(['title', 'year', 'link', 'abstract', 'doi', 'journal', 'authors',
       'author_keywords', 'pubmed_id', 'document_type', 'source'],
      dtype='object')

### read in scopus data


In [9]:
df_scopus = pd.read_parquet(
    "../data/processed/scopus/scopus_data_processed_doisfetched.parquet"
)
df_scopus.head(2)

,authors,title,year,cited_by,doi,link,abstract,author_keywords,index_keywords,references,pubmed_id,document_type,source,eid
0,"[Kiraly; M., Ma; R.C., Dun; N.J.]",Serotonin mediates a slow excitatory potential...,1983,12,10.1016/0006-8993(83)91002-8,https://www.scopus.com/inward/record.uri?eid=2...,Neuropharmacological and histochemical evidenc...,celiac ganglia; non-cholinergic EPSP; serotonin,Action Potentials; Animal; Cyproheptadine; Ele...,"Aghaganian, Asher, Histochemical flourescence ...",6605177,Article,Scopus,2-s2.0-0020509808
1,"[Johnston; C.A., Moore; K.E.]",Measurement of 5-hydroxytryptamine synthesis a...,1983,18,10.1007/BF01250047,https://www.scopus.com/inward/record.uri?eid=2...,High performance liquid chromatography coupled...,None,Animal; Brain; Corpus Striatum; Hydrazines; Hy...,"Brownstein M.J., Palkovits M., Tappaz M.L., Sa...",6619830,Article,Scopus,2-s2.0-0020555550


## Check overlap on DOIs


In [10]:
dois_pm = set(df_pubmed["doi"].to_list())

dois_sc = set(df_scopus["doi"].to_list())

df_pubmed_excl = df_pubmed[~df_pubmed["doi"].isin(dois_sc)]
len(df_pubmed_excl)
print("Number of papers based on doi that are only in PubMed: ", len(df_pubmed_excl))

Number of papers based on doi that are only in PubMed:  1445


## Merge dataframes


In [11]:
# merge dataframes
df_merged = pd.concat([df_scopus, df_pubmed], axis=0, ignore_index=True)

print(df_merged.shape)

(42254, 15)


# remove duplicates

keep title abstract from pubmed if not available scopus is not available


In [12]:
# Define an aggregation function
def aggregate_rows(group):
    aggregated_row = {}
    for column in group.columns:
        if column in ["title", "abstract", "year", "pubmed_id"]:
            non_na_values = group[column].dropna()
            # Check if there are any non-NA values
            if not non_na_values.empty:
                aggregated_row[column] = non_na_values.iloc[0]
            else:
                aggregated_row[column] = np.nan  # Or any other placeholder you prefer
        else:
            # For other data, keep the first value
            aggregated_row[column] = group.iloc[0][column]
    return pd.Series(aggregated_row)


# Apply the function to each group
df_deduplicated = df_merged.groupby("doi").apply(aggregate_rows).reset_index(drop=True)
df_deduplicated.reset_index(drop=True, inplace=True)

df_deduplicated.shape

(35153, 15)

### nameyear


##### first author


In [13]:
def process_author(x):
    # Check if x is a numpy array and has at least one element
    if isinstance(x, np.ndarray) and len(x) > 0 and x[0] is not None:
        return x[0].split(";")[0].strip()
    else:
        return "NoAuth"


# Assign the results to a new column 'first_author'
df_deduplicated["first_author"] = df_deduplicated["authors"].apply(process_author)

# Count the occurrences of each first author
df_deduplicated["first_author"].value_counts(dropna=False).head(10)

first_author
Wang      243
Li        191
Chen      182
Zhang     156
Kim       125
NoAuth    119
Lee       118
Liu       102
Wu         76
Zhou       75
Name: count, dtype: int64

##### date


In [14]:
df_deduplicated["year"].fillna(9999, inplace=True)
# make year an integer columns
df_deduplicated["year"] = df_deduplicated["year"].astype(int)

##### nameyear


In [15]:
# Concatenate 'first_author' and 'year' columns to create a new 'nameyear' column
df_deduplicated["nameyear"] = df_deduplicated["first_author"] + df_deduplicated[
    "year"
].astype(str)

# Create a new column 'unique_nameyear' that counts the cumulative number of each 'nameyear' value
df_deduplicated["unique_nameyear"] = df_deduplicated.groupby("nameyear").cumcount()

# Modify 'unique_nameyear' to append the count to 'nameyear' if the count is greater than 0
df_deduplicated["unique_nameyear"] = df_deduplicated.apply(
    lambda x: f"{x['nameyear']}_{x['unique_nameyear']}"
    if x["unique_nameyear"] > 0
    else x["nameyear"],
    axis=1,
)

### title abstract


In [16]:
df_deduplicated["title_abstract"] = (
    df_deduplicated["title"] + ". " + df_deduplicated["abstract"]
)

for t, a in zip(df_deduplicated["title"], df_deduplicated["abstract"]):
    if t is None:
        print(a)

In [17]:
df_deduplicated["title_abstract"] = (
    df_deduplicated["title"].fillna("NoTitle").astype(str)
    + ". "
    + df_deduplicated["abstract"].fillna("NoAbstract").astype(str)
)

In [25]:
print(
    f"Number of papers without abstract: {df_deduplicated[df_deduplicated['title_abstract'].apply(lambda x: x.endswith('NoAbstract'))].shape[0]}"
)

Number of papers without abstract: 2698


##### paper_id


In [26]:
# sort after year
df_deduplicated.sort_values(by="year", inplace=True)
# reset index
df_deduplicated.reset_index(drop=True, inplace=True)

# creaate paper id
df_deduplicated["paper_id"] = df_deduplicated.index + 1

## save the merged dataframe


In [29]:
### save data
df_deduplicated.to_pickle("../data/interim/merged/merged_data.pkl")